In [18]:
from school_analysis.preprocessing.load import Loader
import pandas as pd

loader = Loader()

def load_special_edu_support_percentage():
    melted_df = loader.load('students_with_special_educational_support_no_gender')

    melted_df['year'] = pd.to_datetime(melted_df['year'], format='%Y')
    melted_df['total'] = pd.to_numeric(melted_df['total'], errors='coerce')
    total_over_years = melted_df.groupby(['year'])['total'].sum()

    children_type = loader.load('school-children-by-type')
    children_type = children_type.rename(columns={'Year': 'year'})
    children_type = children_type.rename(columns={'School Type': 'school'})
    children_type['year'] = pd.to_datetime(children_type['year'], format='%Y')
    
    child_amount_per_year = children_type.groupby(['year'])['Value'].sum()


    merged_df = pd.merge(total_over_years, child_amount_per_year,on=['year'])
    merged_df['edu_support_relative'] = merged_df['total']/merged_df['Value'] 
    return merged_df

def load_grades():
    df_melted = loader.load("abi-grades")
    grades_temp = df_melted.T.reset_index().rename(columns={"level_0": "Federal State", "level_1": "year"})
    df_melted = pd.melt(grades_temp, id_vars=["Federal State", "year"], value_vars=grades_temp.columns[2:], value_name="Value")
    df_melted['year'] = pd.to_datetime(df_melted['year'], format='%Y')
    return df_melted

grades = load_grades()
special_edu_support = load_special_edu_support_percentage()

grades


,Federal State,year,Grade,Value
0,BW,2007-01-01,1.0,714.0
1,BW,2008-01-01,1.0,731.0
2,BW,2009-01-01,1.0,767.0
3,BW,2010-01-01,1.0,822.0
4,BW,2011-01-01,1.0,591.0
...,...,...,...,...
7931,TH,2018-01-01,4.0,1.0
7932,TH,2019-01-01,4.0,0.0
7933,TH,2020-01-01,4.0,0.0
7934,TH,2021-01-01,4.0,0.0


In [19]:
#we just can correlate over the years, because special educational support by states such as school type for the grades is not available
merged_df = pd.merge(grades, special_edu_support, on=['year'])
correlation_coefficient = merged_df['edu_support_relative'].corr(merged_df['Grade'])
correlation_coefficient

1.1525058523412577e-16